In [1]:
#colab連上google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!mkdir -p /drive
#umount /drive
!mount --bind /content/drive/My\ Drive /drive
!mkdir -p /drive/ngrok-ssh
!mkdir -p ~/.ssh


#下載安裝ngrok
!mkdir -p /drive/ngrok-ssh
%cd /drive/ngrok-ssh
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip
!cp /drive/ngrok-ssh/ngrok /ngrok
!chmod +x /ngrok

# 和 ngrok 帳戶進行綁定
!/ngrok authtoken 2gggOkRLOJqcQwoD2JmfggtFga4_byHnwfLy8YgmLXQaLJHm

#安裝pyngrok
!pip install pyngrok

#安裝line bot
!pip install line-bot-sdk

Mounted at /content/drive
/drive/ngrok-ssh
--2024-06-25 05:04:19--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 54.161.241.46, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  39.4MB/s    in 0.3s    

2024-06-25 05:04:19 (39.4 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.4/758.4 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 13.0 MB/s eta 0:00:00


In [2]:
import getpass
from pyngrok import ngrok, conf


#輸入 ngrok token 並與 ngrok 進行認證。
print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth")
conf.get_default().auth_token = getpass.getpass()

# Open a TCP ngrok tunnel to the SSH server
connection_string = ngrok.connect("22", "tcp").public_url

ssh_url, port = connection_string.strip("tcp://").split(":")
print(f" * ngrok tunnel available, access with `ssh root@{ssh_url} -p{port}`")

#2gggOkRLOJqcQwoD2JmfggtFga4_byHnwfLy8YgmLXQaLJHm

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth
··········


 * ngrok tunnel available, access with `ssh root@0.tcp.ngrok.io -p15859`


In [7]:
"""
from flask import Flask, request
from pyngrok import ngrok
import joblib
import pandas as pd
import json
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage, TemplateSendMessage, ButtonsTemplate, PostbackAction, PostbackEvent

app = Flask(__name__)
port = 5000

# 開啟tunnel給server
public_url = ngrok.connect(port).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\" ")

access_token = 'hr57G9x2N1RzDTLfuTmZcmEGbp4NOE+RhKy9bpYVAmJh8kkdmOxjjI2XyyJ1pL2G7fuT0dFbzu7tN/9CXcZtJM6r9n7XLE5ftm04RyX1mNoQ36SLxG+aZo6cGOQe0RUNcsPDNT2Jr+/0THQ4SboBuwdB04t89/1O/w1cDnyilFU='
secret = '55bd2bf40d93d598909c39d480562ddd'

line_bot_api = LineBotApi(access_token)  # token 確認
handler = WebhookHandler(secret)      # secret 確認

model = joblib.load('/content/drive/My Drive/diabete_prediction_model.pkl')


user_state = {}
questions = [
    "請輸入性別 (Male:0 /Female:1 /Other:2): ",
    "請輸入年齡: ",
    "請輸入高血壓狀況 (無:0/有:1): ",
    "請輸入心臟病狀況 (無:0/有:1): ",
    "請輸入吸煙習慣 (never:0 /former:1 /not current:2 /current:3 /ever:4): ",
    "請輸入BMI: ",
    "請輸入HbA1c水平: ",
    "請輸入血糖水平: "
]


def validate_input(step, input_value):
    try:
        value = float(input_value)
        if value < 0:
            return False, "不能輸入負數，請重新輸入。"

        if step == 0 and value not in [0, 1, 2]:
            return False, "性別只能輸入0, 1, 2，請重新輸入。"
        elif step == 1 and (value < 0 or value > 120):
            return False, "年齡必須在0到120之間，請重新輸入。"
        elif step == 2 and value not in [0, 1]:
            return False, "高血壓狀況只能輸入0或1，請重新輸入。"
        elif step == 3 and value not in [0, 1]:
            return False, "心臟病狀況只能輸入0或1，請重新輸入。"
        elif step == 4 and value not in [0, 1, 2, 3, 4]:
            return False, "吸煙習慣只能輸入0, 1, 2, 3, 4，請重新輸入。"
        elif step in [5, 6, 7] and value < 0:
            return False, "BMI, HbA1c水平和血糖水平不能為負，請重新輸入。"
        return True, value
    except ValueError:
        return False, "請輸入數字。"

@app.route("/", methods=['POST'])
def linebot():
    body = request.get_data(as_text=True)
    try:
        json_data = json.loads(body)
        signature = request.headers['X-Line-Signature']
        handler.handle(body, signature)
        user_id = json_data['events'][0]['source']['userId']
        tk = json_data['events'][0]['replyToken']

        if 'postback' in json_data['events'][0]:
            data = json_data['events'][0]['postback']['data']
            if user_id not in user_state:
                user_state[user_id] = {'step': -2, 'data': []}

            if data == 'action=yes':
                user_state[user_id]['step'] = 0
                line_bot_api.reply_message(tk, TextSendMessage(text=questions[0]))
            elif data == 'action=no':
                line_bot_api.reply_message(tk, TextSendMessage(text="好的，謝謝光臨，有需要再叫我喔"))
                del user_state[user_id]
            return 'OK'

        msg = json_data['events'][0]['message']['text']

        if user_id not in user_state:
            user_state[user_id] = {'step': -2, 'data': []}
            introduction = ("您好，我是健康智能管家。\n"
                            "也是糖尿病預測專家，\n"
                            "您可以叫我阿瑄=U=\n"
                            "請問是否進行糖尿病預測呢?")
            buttons_template = TemplateSendMessage(
                alt_text='是否進行糖尿病預測？',
                template=ButtonsTemplate(
                    title='是否進行糖尿病預測？',
                    text=introduction,
                    actions=[
                        PostbackAction(label='是', data='action=yes'),
                        PostbackAction(label='否', data='action=no')
                    ]
                )
            )
            line_bot_api.reply_message(tk, buttons_template)
            return 'OK'

        step = user_state[user_id]['step']

        if step >= 0:
            valid, response = validate_input(step, msg)

            if valid:
                user_state[user_id]['data'].append(response)
                if step < len(questions) - 1:
                    user_state[user_id]['step'] += 1
                    line_bot_api.reply_message(tk, TextSendMessage(text=questions[step + 1]))
                else:
                    user_input = pd.DataFrame([user_state[user_id]['data']], columns=[
                        'gender', 'age', 'hypertension', 'heart_disease', 'smoking_history',
                        'bmi', 'HbA1c_level', 'blood_glucose_level'
                    ])
                    prediction = model.predict(user_input)
                    result = "沒有糖尿病" if prediction[0] == 0 else "有糖尿病"
                    line_bot_api.reply_message(tk, [
                        TextSendMessage(text=f"結果：{result}"),
                        TextSendMessage(text="有需要再呼叫我喔!!!")
                    ])
                    del user_state[user_id]
            else:
                error_message = response
                line_bot_api.reply_message(tk, [
                    TextSendMessage(text=error_message),
                    TextSendMessage(text=questions[step])
                ])

    except Exception as e:
        print(f"Error: {e}")
        try:
            tk = json_data['events'][0]['replyToken']
            line_bot_api.reply_message(tk, TextSendMessage(text="沙織聽不懂ㄟ"))
        except Exception as inner_e:
            print(f"Inner Error: {inner_e}")

    return 'OK'

if __name__ == "__main__":
    app.run(port=port)

"""





from flask import Flask, request, abort
from pyngrok import ngrok
import joblib
import pandas as pd
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage, TemplateSendMessage, ButtonsTemplate, PostbackAction, PostbackEvent

app = Flask(__name__)
port = 5000

# Set up ngrok tunnel to expose local server
public_url = ngrok.connect(port).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\" ")

# LINE Messaging API credentials
access_token = 'hr57G9x2N1RzDTLfuTmZcmEGbp4NOE+RhKy9bpYVAmJh8kkdmOxjjI2XyyJ1pL2G7fuT0dFbzu7tN/9CXcZtJM6r9n7XLE5ftm04RyX1mNoQ36SLxG+aZo6cGOQe0RUNcsPDNT2Jr+/0THQ4SboBuwdB04t89/1O/w1cDnyilFU='
secret = '55bd2bf40d93d598909c39d480562ddd'

line_bot_api = LineBotApi(access_token)  # token 確認
handler = WebhookHandler(secret)      # secret 確認

model = joblib.load('/content/drive/My Drive/diabete_prediction_model.pkl')

# User state management
user_state = {}

# Text questions (4-7)
questions = [
    "請輸入年齡: ",
    "請輸入BMI: ",
    "請輸入HbA1c水平: ",
    "請輸入血糖水平: "
]

# Button questions (0-3)
button_questions = {
    0: ('性別', [
        ('男', '0'),
        ('女', '1'),
        ('其他', '2')
    ]),
    1: ('高血壓狀況', [
        ('有', '1'),
        ('無', '0')
    ]),
    2: ('心臟病狀況', [
        ('有', '1'),
        ('無', '0')
    ]),
    3: ('吸煙習慣', [
        ('ever', '4'),
        ('current', '3'),
        ('former', '1'),
        ('never', '0')
    ])
}

@app.route("/", methods=['POST'])
def webhook():
    body = request.get_data(as_text=True)
    signature = request.headers['X-Line-Signature']
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_text_message(event):
    user_id = event.source.user_id
    msg = event.message.text
    if user_id not in user_state:
        user_state[user_id] = {'step': -2, 'data': []}

        introduction = ("您好，我是健康智能管家。\n"
                 "也是糖尿病預測專家，\n"
                 "您可以叫我阿瑄=U=\n"
                 "請問是否進行糖尿病預測呢?")

        buttons_template = TemplateSendMessage(
            alt_text='是否進行糖尿病預測？',
            template=ButtonsTemplate(
                title='是否進行糖尿病預測？',
                text=introduction,
                actions=[
                    PostbackAction(label='是', data='action=yes'),
                    PostbackAction(label='否', data='action=no')
                ]
            )
        )

        line_bot_api.reply_message(event.reply_token, buttons_template)
        return

    step = user_state[user_id]['step']
    if step == -2:
        line_bot_api.reply_message(event.reply_token, TextSendMessage(text="請選擇按鈕選項。"))
        return

    if step < 4:
        line_bot_api.reply_message(event.reply_token, TextSendMessage(text="請選擇按鈕選項。"))
        return

    current_question_index = step - 4
    try:
        if current_question_index >= 0:
            float_msg = float(msg)
            if float_msg < 0: #負數偵錯
                line_bot_api.reply_message(
                    event.reply_token,
                    TextSendMessage(text="請輸入大於等於零的有效數字")
                )
                return
            user_state[user_id]['data'].append(float_msg)
            step += 1
            user_state[user_id]['step'] = step
            if step < len(questions) + 4:
                if step < 8:
                    line_bot_api.reply_message(
                        event.reply_token,
                        TextSendMessage(text=questions[current_question_index + 1])
                    )
                else:
                    process_final_input(event.reply_token, user_id)
            else:
                process_final_input(event.reply_token, user_id)
        else:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text="請選擇按鈕選項。")
            )
    except ValueError:
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text="請輸入有效數字")
        )

def process_final_input(reply_token, user_id):
    user_data = user_state[user_id]['data']
    user_input = pd.DataFrame([user_data], columns=[
        'gender', 'age', 'hypertension', 'heart_disease', 'smoking_history',
        'bmi', 'HbA1c_level', 'blood_glucose_level'
    ])
    prediction = model.predict(user_input)
    result = "没有糖尿病" if prediction[0] == 0 else "有糖尿病"
    line_bot_api.reply_message(reply_token, [
        TextSendMessage(text=f"預測结果：{result}"),
        TextSendMessage(text="謝謝光臨!! 有需要都可以在叫我喔")
    ])
    del user_state[user_id]

@handler.add(PostbackEvent)
def handle_postback(event):
    user_id = event.source.user_id
    data = event.postback.data

    if user_id not in user_state:
        user_state[user_id] = {'step': -2, 'data': []}

    if data == 'action=yes':
        user_state[user_id] = {'step': 0, 'data': []}
        send_button_template(event.reply_token, 0)
    elif data == 'action=no':
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text="謝謝光臨!! 有需要都可以在叫我喔")
        )
        del user_state[user_id]
    else:
        step = user_state[user_id]['step']
        user_state[user_id]['data'].append(int(data))
        step += 1
        user_state[user_id]['step'] = step
        if step < 4:
            send_button_template(event.reply_token, step)
        elif step == 4:
            line_bot_api.reply_message(event.reply_token, TextSendMessage(text=questions[0]))
        else:
            # Handle the next text question
            current_question_index = step - 4
            line_bot_api.reply_message(event.reply_token, TextSendMessage(text=questions[current_question_index]))

def send_button_template(reply_token, step):
    title, choices = button_questions[step]
    actions = [PostbackAction(label=label, data=data) for label, data in choices]
    template_message = TemplateSendMessage(
        alt_text=f'{title}',
        template=ButtonsTemplate(
            title=title, text=f'請選擇您的{title}', actions=actions
        )
    )
    line_bot_api.reply_message(reply_token, template_message)
    step

if __name__ == "__main__":
    app.run(port=port)


 * ngrok tunnel "https://b97b-34-145-214-89.ngrok-free.app" -> "http://127.0.0.1:5000" 


<ipython-input-7-513bd1bab9b0>:174: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(access_token)  # token 確認
<ipython-input-7-513bd1bab9b0>:175: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler(secret)      # secret 確認


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [25/Jun/2024 05:08:02] "POST / HTTP/1.1" 200 -
<ipython-input-7-513bd1bab9b0>:247: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(event.reply_token, buttons_template)
INFO:werkzeug:127.0.0.1 - - [25/Jun/2024 05:08:12] "POST / HTTP/1.1" 200 -
<ipython-input-7-513bd1bab9b0>:319: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sd

In [ ]:
~